In [1]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import time
import warnings
import os
from tiki.mini import TikiMini

warnings.filterwarnings('ignore', category=UserWarning)
os.environ['GST_DEBUG'] = '0'

# 로봇 초기화
tiki = TikiMini()
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)
print("로봇 초기화 완료")

# 주행 파라미터
base_speed = 70
max_steering = 25
frame_center_x = 160  # 320x240 기준 중앙 X

# PID 제어
class PIDController:
    def __init__(self, kp=0.6, ki=0.0, kd=0.15):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0
        self.prev_error = 0

    def compute(self, error):
        self.integral += error
        derivative = error - self.prev_error
        self.prev_error = error
        self.integral = np.clip(self.integral, -200, 200)
        return self.kp * error + self.ki * self.integral + self.kd * derivative

pid = PIDController()

# 카메라 파이프라인
pipeline = (
    "nvarguscamerasrc ! "
    "video/x-raw(memory:NVMM), width=320, height=240, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! "
    "video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
if not cap.isOpened():
    raise RuntimeError("카메라를 열 수 없습니다.")

video_widget = widgets.Image(format='jpeg',
                             layout=widgets.Layout(width='640px', height='480px'))
display(video_widget)

# -----------------------
# Bird's-Eye View 설정
# -----------------------
bird_width, bird_height = 320, 240

src_points = np.float32([
    [60, 120],
    [260, 120],
    [310, 230],
    [10, 230],
])

dst_points = np.float32([
    [0, 0],
    [bird_width - 1, 0],
    [bird_width - 1, bird_height - 1],
    [0, bird_height - 1],
])

M = cv2.getPerspectiveTransform(src_points, dst_points)

def frame_to_bytes(frame):
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 45])
    return buf.tobytes()


# ============================================================
#   1) 슬라이딩 윈도우 + 시각화
# ============================================================
def sliding_window_center_with_vis(binary_mask, num_windows=8, margin=20, min_pixels=30):
    h, w = binary_mask.shape[:2]
    window_height = h // num_windows
    ys, xs = np.where(binary_mask == 255)

    vis = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)

    if len(xs) == 0:
        return None, vis

    bottom_thresh = int(h * 0.75)
    bottom_inds = ys >= bottom_thresh
    if np.any(bottom_inds):
        current_x = int(np.mean(xs[bottom_inds]))
    else:
        current_x = int(np.mean(xs))

    centers = []
    pts = []

    for i in range(num_windows):
        y_low = h - (i+1)*window_height
        y_high = h - i*window_height
        y_low = max(0, y_low)
        if y_high <= y_low: continue

        inds = (ys >= y_low) & (ys < y_high)
        win_xs = xs[inds]
        if len(win_xs) == 0: continue

        x_min = max(0, current_x - margin)
        x_max = min(w - 1, current_x + margin)

        lane_inds = (win_xs >= x_min) & (win_xs <= x_max)
        if np.sum(lane_inds) < min_pixels:
            continue

        current_x = int(np.mean(win_xs[lane_inds]))
        centers.append(current_x)

        cy = (y_low + y_high) // 2
        pts.append((current_x, cy))

        cv2.rectangle(vis, (x_min, y_low), (x_max, y_high), (0,255,0), 1)
        cv2.circle(vis, (current_x, cy), 3, (255,0,255), -1)

    if len(centers) == 0:
        return int(np.mean(xs)), vis

    pts_np = np.array(pts, np.int32)
    cv2.polylines(vis, [pts_np], False, (255,0,255), 2)

    return int(np.mean(centers)), vis


# ============================================================
#   2) 중앙 흰색 라인 + 모든 디버그뷰 생성
# ============================================================
def detect_lane_center(frame_bgr, white_threshold=180):
    h, w = frame_bgr.shape[:2]

    # ROI (하단부)
    roi_y1 = int(h*0.60)
    roi_y2 = h
    roi_x1 = int(w*0.10)
    roi_x2 = int(w*0.90)
    roi = frame_bgr[roi_y1:roi_y2, roi_x1:roi_x2]
    roi_h, roi_w = roi.shape[:2]

    # -----------------------------
    # ① BGR -> HSV 변환
    # -----------------------------
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # 흰색만 남기기 위한 조건:
    #   - V(밝기) : 충분히 밝음
    #   - S(채도) : 낮음 (노란색은 채도가 높아서 걸러짐)
    # 값은 상황 보고 조절: S_MAX, V_MIN
    S_MAX = 60   # 채도 상한 (줄이면 노란색 더 잘 제거)
    V_MIN = 180  # 밝기 하한 (줄이면 어두운 흰색도 포함)

    lower_white = np.array([0, 0, V_MIN], dtype=np.uint8)
    upper_white = np.array([180, S_MAX, 255], dtype=np.uint8)
    white_mask = cv2.inRange(hsv, lower_white, upper_white)

    # -----------------------------
    # ② 필요하면 노란색 범위 명시적으로 제거
    #    (HSV에서 노란색 H는 대략 15~35 근처)
    # -----------------------------
    # 이 부분은 상황 봐서 켜거나/끄면 됨
    lower_yellow = np.array([15, 80, 80], dtype=np.uint8)
    upper_yellow = np.array([40, 255, 255], dtype=np.uint8)
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # 흰색 마스크에서 노란색 부분 빼기
    white_mask = cv2.bitwise_and(white_mask, cv2.bitwise_not(yellow_mask))

    # 약간의 노이즈 제거용 (선택)
    white_mask = cv2.GaussianBlur(white_mask, (5,5), 0)

    ys, xs = np.where(white_mask == 255)
    white_mask_bgr = cv2.cvtColor(white_mask, cv2.COLOR_GRAY2BGR)

    if len(xs) == 0:
        return None, False, {
            "roi": roi,
            "white": white_mask_bgr,
            "sliding": white_mask_bgr.copy()
        }

    # 슬라이딩윈도우
    center_x_rel, sliding_vis = sliding_window_center_with_vis(white_mask)

    if center_x_rel is None:
        center_x_rel = int(np.mean(xs))

    center_x_frame = roi_x1 + center_x_rel

    # 가로 흰줄 판단 (STOP LINE 등)
    x_range = np.max(xs) - np.min(xs)
    y_range = np.max(ys) - np.min(ys)
    is_horizontal_bar = (x_range > roi_w*0.6 and y_range < roi_h*0.25)

    return center_x_frame, is_horizontal_bar, {
        "roi": roi,
        "white": white_mask_bgr,
        "sliding": sliding_vis
    }

# ============================================================
#   3) 2×2 디버그 패널 생성
# ============================================================
def make_panel(bird, dbg):
    def R(img): return cv2.resize(img, (320, 240))

    panel_top = cv2.hconcat([R(bird), R(dbg["roi"])])
    panel_bot = cv2.hconcat([R(dbg["white"]), R(dbg["sliding"])])
    return cv2.vconcat([panel_top, panel_bot])


# ============================================================
#   4) 메인 루프
# ============================================================
try:
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.05)
            continue

        frame = cv2.flip(frame, -1)

        # 버드뷰 생성
        bird = cv2.warpPerspective(frame, M, (bird_width, bird_height))

        # 중앙 흰색라인 검출
        center_x, is_horizontal_bar, dbg = detect_lane_center(bird)

        view = bird.copy()

        if center_x is not None:
            error = center_x - frame_center_x

            steering = pid.compute(error)
            steering *= 0.5
            steering = float(np.clip(steering, -max_steering, max_steering))
            if is_horizontal_bar:
                steering = 0

            L = int(np.clip(base_speed + steering, 0, 127))
            Rm = int(np.clip(base_speed - steering, 0, 127))

            tiki.set_motor_power(tiki.MOTOR_LEFT, L)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, Rm)

        else:
            tiki.stop()

        panel = make_panel(view, dbg)
        video_widget.value = frame_to_bytes(panel)

        time.sleep(0.02)

except KeyboardInterrupt:
    tiki.stop()
    cap.release()
    print("종료!")


로봇 초기화 완료


Error generated. /dvs/git/dirty/git-master_linux/multimedia/nvgstreamer/gst-nvarguscamera/gstnvarguscamerasrc.cpp, execute:751 Failed to create CaptureSession
[ WARN:0@0.922] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Image(value=b'', format='jpeg', layout="Layout(height='480px', width='640px')")

종료!
